Use this to create the database

In [1]:
import os
from os.path import expanduser
import pandas as pd
import pandas.io.sql as pd_sql
from sqlalchemy import create_engine

from functions.data import preprocess
from functions.auth.connections import postgres_connect_fetch_close
from functions.auth.connections import postgres_connection

connection_uri = postgres_connection('sf_rent_petitions')
engine = create_engine(connection_uri)

In [2]:
save=False

# Petitions Data

In [3]:
# Get Data
petition_df = pd.read_csv("data/downloads/Petitions_to_the_Rent_Board.csv")
petition_df = preprocess(petition_df)
petition_df

/home/harrisonized/anaconda3/envs/basic/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date,priority,appeal_id,filing_party,prop_i_petition,ground_landlord_capital_improvement,ground_landlord_comparable_rents,ground_landlord_extension_of_time_for_capital_improvements,ground_landlord_intent_to_withdraw_ellis,ground_landlord_other,...,neighborhood_number,computed_region_fyvs_ahh9,computed_region_p5aj_wyqh,computed_region_rxqg_mtj9,computed_region_yftq_j783,computed_region_bh8s_q3mv,latitude,longitude,linked_petition_ids,num_petition_id
0,03/29/2019,No,NaN,tenant,NaN,False,False,False,False,False,...,2.0,2.0,7.0,5.0,2.0,28859.0,37.745261,-122.421229,"[""T190501""]",1
1,03/29/2019,No,NaN,landlord,NaN,True,False,False,False,False,...,9.0,10.0,9.0,11.0,7.0,28852.0,37.772274,-122.425349,"[""L190451"", ""L190450""]",2
2,03/29/2019,No,NaN,tenant,NaN,False,False,False,False,False,...,32.0,32.0,1.0,10.0,3.0,308.0,37.801896,-122.413271,"[""T190536""]",1
3,03/29/2019,No,NaN,landlord,NaN,False,False,False,False,False,...,35.0,35.0,8.0,3.0,1.0,29491.0,37.748352,-122.503939,"[""L190488""]",1
4,03/29/2019,No,NaN,tenant,NaN,False,False,False,False,False,...,9.0,10.0,9.0,11.0,7.0,28852.0,37.773615,-122.426540,"[""E190473""]",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40744,01/02/1997,No,NaN,tenant,NaN,False,False,False,False,False,...,5.0,3.0,5.0,5.0,15.0,28862.0,37.761205,-122.445514,"[""R004-71E""]",1
40745,01/02/1997,No,NaN,tenant,NaN,False,False,False,False,False,...,21.0,21.0,1.0,10.0,5.0,28857.0,37.790931,-122.409789,"[""R004-86T""]",1
40746,01/02/1997,No,NaN,tenant,NaN,False,False,False,False,False,...,11.0,11.0,6.0,2.0,11.0,54.0,37.776452,-122.460659,"[""R001-72S""]",1
40747,01/02/1997,No,R001-71A,tenant,NaN,False,False,False,False,False,...,13.0,17.0,9.0,1.0,13.0,57.0,37.797053,-122.447018,"[""R004-87T""]",1


In [4]:
if save:
    petition_df.to_csv("data/petitions.csv", index=False)

In [5]:
# Create database
query = """
CREATE DATABASE sf_rent_petitions;
"""
postgres_connect_fetch_close(query, cred, 'postgres', autocommit=True)

In [6]:
# To Postgres
petition_df = pd.read_csv("data/petitions.csv") # Import
petition_df.to_sql('petitions', engine, index=True) # Create database

/home/harrisonized/anaconda3/envs/basic/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
# Check
query = """
SELECT *
FROM petitions
;
"""
petition_df = pd_sql.read_sql(query, connection_uri) # grab data as a dataframe
petition_df = petition_df.reset_index(drop=True)
petition_df.head()

,index,date,priority,appeal_id,filing_party,prop_i_petition,ground_landlord_capital_improvement,ground_landlord_comparable_rents,ground_landlord_extension_of_time_for_capital_improvements,ground_landlord_intent_to_withdraw_ellis,...,neighborhood_number,computed_region_fyvs_ahh9,computed_region_p5aj_wyqh,computed_region_rxqg_mtj9,computed_region_yftq_j783,computed_region_bh8s_q3mv,latitude,longitude,linked_petition_ids,num_petition_id
0,0,03/29/2019,No,None,tenant,None,False,False,False,False,...,2.0,2.0,7.0,5.0,2.0,28859.0,37.745261,-122.421229,"[""T190501""]",1
1,1,03/29/2019,No,None,landlord,None,True,False,False,False,...,9.0,10.0,9.0,11.0,7.0,28852.0,37.772274,-122.425349,"[""L190451"", ""L190450""]",2
2,2,03/29/2019,No,None,tenant,None,False,False,False,False,...,32.0,32.0,1.0,10.0,3.0,308.0,37.801896,-122.413271,"[""T190536""]",1
3,3,03/29/2019,No,None,landlord,None,False,False,False,False,...,35.0,35.0,8.0,3.0,1.0,29491.0,37.748352,-122.503939,"[""L190488""]",1
4,4,03/29/2019,No,None,tenant,None,False,False,False,False,...,9.0,10.0,9.0,11.0,7.0,28852.0,37.773615,-122.426540,"[""E190473""]",1


# Unemployment

In [8]:
unemployment_df = pd.read_csv("data/downloads/CASANF0URN.csv") # Import
unemployment_df = unemployment_df.rename(columns={'DATE': 'date',
                                                  'CASANF0URN': 'unemployment_rate'}) # rename columns

In [9]:
# To Postgres
unemployment_df.to_sql('unemployment', engine, index=True)

In [10]:
# Check
query = """
SELECT *
FROM unemployment
;
"""
unemployment_df = pd_sql.read_sql(query, connection_uri) # grab data as a dataframe
unemployment_df.head()

,index,date,unemployment_rate
0,0,1990-01-01,3.7
1,1,1990-02-01,3.4
2,2,1990-03-01,3.2
3,3,1990-04-01,3.5
4,4,1990-05-01,3.5
